In [48]:
import numpy as np
import Agent
import Environment
import Referee

In [49]:
import pandas as pd

In [50]:
df_type = pd.read_csv('type.csv',index_col=0).T
df_type

type,normal,fire,water,electric,grass,ice,fighting,poison,ground,flying,psychic,bug,rock,ghost,dragon,dark,steel,fairy
normal,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
fire,1.0,0.5,2.0,1.0,0.5,0.5,1.0,1.0,2.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,0.5,0.5
water,1.0,0.5,0.5,2.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0
electric,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0
grass,1.0,2.0,0.5,0.5,0.5,2.0,1.0,2.0,0.5,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
ice,1.0,2.0,1.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0
fighting,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,0.5,0.5,1.0,1.0,0.5,1.0,2.0
poison,1.0,1.0,1.0,1.0,0.5,1.0,0.5,0.5,2.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0,0.5
ground,1.0,1.0,2.0,0.0,2.0,2.0,1.0,0.5,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0,1.0
flying,1.0,1.0,1.0,2.0,0.5,2.0,0.5,1.0,0.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,1.0,1.0


In [51]:
dict_from_move = pd.read_csv('move.csv',encoding="cp932").T.to_dict()
print(dict_from_move)

{0: {'name': 'げきりん', 'priority': 0, 'is_attack': True, 'move_category': 'physical', 'type': 'bug', 'critical_rank': 0, 'accuracy': 100, 'power': 100, 'PP': 10, 'is_combo': False, 'is_contact': True}, 1: {'name': 'じしん', 'priority': 0, 'is_attack': True, 'move_category': 'physical', 'type': 'ground', 'critical_rank': 0, 'accuracy': 100, 'power': 100, 'PP': 10, 'is_combo': False, 'is_contact': False}, 2: {'name': 'アイアンヘッド', 'priority': 0, 'is_attack': True, 'move_category': 'physical', 'type': 'steel', 'critical_rank': 0, 'accuracy': 100, 'power': 80, 'PP': 15, 'is_combo': False, 'is_contact': True}, 3: {'name': 'ねごと', 'priority': 0, 'is_attack': False, 'move_category': 'support', 'type': 'normal', 'critical_rank': 0, 'accuracy': 0, 'power': 0, 'PP': 10, 'is_combo': False, 'is_contact': True}, 4: {'name': 'ゴールドラッシュ', 'priority': 0, 'is_attack': True, 'move_category': 'special', 'type': 'steel', 'critical_rank': 0, 'accuracy': 100, 'power': 120, 'PP': 5, 'is_combo': False, 'is_contact': Fa

In [54]:
# 初期化

state = {
    # 処理に必要な情報
    'system_info': {
        # 現在のフェーズ
        'phase': 'phase__battle_start',
        # 現在のプロセス
        'process': '',
        # 処理順更新フラグ
        'update_battle_order_flag': False,
        # 現在のフェーズ処理対象
        'phase_target': 'A',
        # 現在のプロセス処理対象
        'process_target': 'A',
        # 処理順
        'battle_order': ['A', 'B'],
        # 行動順
        'move_order': [],
        # 計算したダメージ量
        'damage': 0
    },
    # 共通の情報
    'all_info':{
        'weather': ''
    },
    # A（自分）の情報
    'A_info': {
        # 交代待機がいない場合は -1
        'activate_pokemon_id': 0,
        # 場に誰も出ていない場合は -1
        'active_pokemon_id': -1,
        # 場に出てこれないポケモンがいない場合は、-1
        'disable_activate_pokemon_id': -1,
        # テラスタルを使用したかどうか
        'tera_orb': 'unused',
        # 行動可能かどうか
        'is_actionable': True,
        # 選択された技（選択されていない場合は-1）
        'selected_move_id': -1,
        # テラスタル化する選択をしたか
        'selected_tera': False,
        # 場に出ているポケモンが状態異常「ひんし」になった時刻
        'fainted_time': np.inf,
        'pokemons': {
            0 :{
                'name': 'ガブリアス',
                'ailment': '',
                'conditions': {},
                'is_tera': False,
                'level': 50,
                'types': ['dragon', 'ground'],
                # 'ability': "さめはだ",
                'ability': "いかく",
                'tera_type': 'ground',
                # テラスタル化していて、かつ元タイプと一致しているかどうか
                'tera_type_match': False,
                'item': 'こだわりハチマキ',
                'max_H': 183,
                'H': 183,
                'A': int(200 * 1.5),
                'B': 116,
                'C': 90,
                'D': 105,
                'S': 154,
                'A_rank': 0,
                'critical_rank': 0,
                'accuracy_rank': 0,
                'moves': {
                    0:dict_from_move[0], #げきりん
                    1:dict_from_move[1], #じしん
                    2:dict_from_move[2], #アイアンヘッド
                    3:dict_from_move[3]  #ねごと
                }
            },
            1: {
                'name': 'サーフゴー',
                'ailment': '',
                'conditions': {},
                'is_tera': False,
                'level': 50,
                'types': ['ghost', 'steel'],
                'ability': "おうごんのからだ",
                'tera_type': 'flying',
                # テラスタル化していて、かつ元タイプと一致しているかどうか
                'tera_type_match': False,
                'item': 'こだわりスカーフ',
                #'item': 'こだわりスカーフ',
                'max_H': 167,
                'H': 163,
                'A': 58,
                'B': 115,
                'C': 185,
                'D': 111,
                'S': 149,
                'A_rank': 0,
                'critical_rank': 0,
                'accuracy_rank': 0,
                'moves': {
                    0:dict_from_move[4], #ゴールドラッシュ
                    1:dict_from_move[5], #シャドーボール
                    2:dict_from_move[6], #トリック
                    3:dict_from_move[7]  #パワージェム
                }
            },
            2: {
                'name': 'ドドゲザン',
                'ailment': '',
                'conditions': {},
                'is_tera': False,
                'level': 50,
                'types': ['dark', 'steel'],
                'ability': "まけんき",
                'tera_type': 'dark',
                # テラスタル化していて、かつ元タイプと一致しているかどうか
                'tera_type_match': True,
                'item': '',
                #'item': 'とつげきチョッキ',
                'max_H': 207,
                'H': 207,
                'A': 205,
                'B': 140,
                'C': 65,
                'D': 105,
                'S': 71,
                'A_rank': 0,
                'critical_rank': 0,
                'accuracy_rank': 0,
                'moves': {
                    0:dict_from_move[8], #ふいうち
                    1:dict_from_move[9], #ドゲザン
                    2:dict_from_move[2], #アイアンヘッド
                    3:dict_from_move[10] #かわらわり
                }
            }
        }
    },
    # B（相手）の情報
    'B_info': {
        # 交代待機がいない場合は -1
        'activate_pokemon_id': 0,
        # 場に誰も出ていない場合は -1
        'active_pokemon_id': -1,
        # 場に出てこれないポケモンがいない場合は、-1
        'disable_activate_pokemon_id': -1,
        # テラスタルを使用したかどうか
        'tera_orb': 'unused',
        # 行動可能かどうか
        'is_actionable': True,
        # 選択された技（選択されていない場合は-1）
        'selected_move_id': -1,
        # テラスタル化する選択をしたか
        'selected_tera': False,
        # 場に出ているポケモンが状態異常「ひんし」になった時刻
        'fainted_time': np.inf,
        'pokemons': {
            0 :{
                'name': 'ミミッキュ',
                'ailment': '',
                'conditions': {},
                'is_tera': False,
                'level': 50,
                'types': ['ghost', 'fairy'],
                'ability': 'ばけのかわ',
                'tera_type': 'fairy',
                # テラスタル化していて、かつ元タイプと一致しているかどうか
                'tera_type_match': False,
                # 'item': 'いのちのたま',
                'item': '',
                'max_H': 131,
                'H': 131,
                'A': 156,
                'B': 100,
                'C': 63,
                'D': 125,
                'S': 148,
                'A_rank': 0,
                'critical_rank': 0,
                'accuracy_rank': 0,
                'moves': {
                    0:dict_from_move[11], #じゃれつく
                    1:dict_from_move[12], #シャドークロー
                    2:dict_from_move[13], #かげうち
                    3:dict_from_move[14]  #つるぎのまい
                }
            },
            1: {
                'name': 'ギャラドス',
                'ailment': '',
                'conditions': {},
                'is_tera': False,
                'level': 50,
                'types': ['water', 'flying'],
                # 'ability': "じしんかじょう",
                'ability': "いかく",
                'tera_type': 'flying',
                # テラスタル化していて、かつ元タイプと一致しているかどうか
                'tera_type_match': True,
                'item': '',
                #'item': 'ラムのみ',
                'max_H': 171,
                'H': 171,
                'A': 177,
                'B': 99,
                'C': 72,
                'D': 120,
                'S': 146,
                'A_rank': 0,
                'critical_rank': 0,
                'accuracy_rank': 0,
                'moves': {
                    0:dict_from_move[15], #テラバースト
                    1:dict_from_move[1], #じしん
                    2:dict_from_move[16], #りゅうのまい
                    3:dict_from_move[17]  #ちょうはつ
                }
            },
            2: {
                'name': 'セグレイブ',
                'ailment': '',
                'conditions': {},
                'is_tera': False,
                'level': 50,
                'types': ['dragon', 'ice'],
                # 'ability': "アイスボディ",
                'ability': "ねつこうかん",
                'tera_type': 'ground',
                # テラスタル化していて、かつ元タイプと一致しているかどうか
                'tera_type_match': False,
                #'item': 'いかさまダイス',
                'item': '',
                'max_H': 191,
                'H': 191,
                'A': 197,
                'B': 112,
                'C': 60,
                'D': 106,
                'S': 152,
                'A_rank': 0,
                'critical_rank': 0,
                'accuracy_rank': 0,
                'moves': {
                    0:dict_from_move[18], #つららばり
                    1:dict_from_move[1], #じしん
                    2:dict_from_move[19], #こおりのつぶて
                    3:dict_from_move[15]  #テラバースト
                }
            }
        }
    }
}

action = -1

In [55]:
# 対戦ループ

for _ in range(100):
    # エージェントの行動と現在の状態から、次の状態を決定
    state, request = Environment.update_state(state, action)


    # エージェントは現在の状態から、行動を決定
    if request == 'A':
        action = Agent.response_action(state, request)
    elif request == 'B':
        action = Agent.response_action(state, request, auto=True)

    # 終了判定
    elif request == 'end_judgement':
        if Referee.judge_end(state):
            break
    else:
        Exception
    
    # デバッグ用
    if action == 'debug':
        break

comment: 対戦開始！
comment: プレイヤーA は ガブリアス を繰り出した！
comment: プレイヤーB は ミミッキュ を繰り出した！
comment: プレイヤーB の ミミッキュ は いかく されて 攻撃が １だんかい　さがった。

----------現在の状態----------
-----全体-----
天候: 

-----プレイヤーA-----
ポケモン	: ガブリアス (Lv:50)
タイプ	: ['dragon', 'ground']
ﾃﾗｽﾀﾙ化	: False
HP	: 183/183
状態異常	: 

控え匹数	: 2

-----プレイヤーB-----
ポケモン	: ミミッキュ (Lv:50)
タイプ	: ['ghost', 'fairy']
ﾃﾗｽﾀﾙ化	: False
HP	: 1310/131
状態異常	: 

控え匹数	: 2


----------
プレイヤーA の行動を選択してください(半角英数字記号)：
テラスタル化したい場合は、選択肢の後ろに「_tera」と入力してください

{'move_0': 'げきりん', 'move_1': 'じしん', 'move_2': 'アイアンヘッド', 'move_3': 'ねごと'}
{'pokemon_1': 'サーフゴー', 'pokemon_2': 'ドドゲザン'}

comment: プレイヤーB の ミミッキュ の かげうち ！
comment: プレイヤーA の ガブリアス の げきりん ！
comment: 効果は いまひとつ のようだ！
comment: プレイヤーB の ミミッキュ の かげうち ！
comment: プレイヤーA の ガブリアス の げきりん ！
comment: 効果は いまひとつ のようだ！
comment: プレイヤーA の ガブリアス は つかれはてて こんらん した！

----------現在の状態----------
-----全体-----
天候: 

-----プレイヤーA-----
ポケモン	: ガブリアス (Lv:50)
タイプ	: ['dragon', 'ground']
ﾃﾗｽﾀﾙ化	: False
HP	: 137/183
状態異常	: 

控え匹数	: 2

-----プレイヤーB-----
ポケモ

In [41]:
state['A_info']

{'activate_pokemon_id': -1,
 'active_pokemon_id': 0,
 'disable_activate_pokemon_id': -1,
 'tera_orb': 'unused',
 'is_actionable': True,
 'selected_move_id': -1,
 'selected_tera': False,
 'fainted_time': inf,
 'pokemons': {0: {'name': 'ガブリアス',
   'ailment': '',
   'conditions': {},
   'is_tera': False,
   'level': 50,
   'types': ['dragon', 'ground'],
   'ability': 'いかく',
   'tera_type': 'ground',
   'tera_type_match': False,
   'item': 'こだわりハチマキ',
   'max_H': 183,
   'H': 183,
   'A': 300,
   'B': 116,
   'C': 90,
   'D': 105,
   'S': 154,
   'A_rank': 0,
   'critical_rank': 0,
   'accuracy_rank': 0,
   'moves': {0: {'name': 'げきりん',
     'priority': 0,
     'is_attack': True,
     'move_category': 'physical',
     'type': 'bug',
     'critical_rank': 0,
     'accuracy': 100,
     'power': 100,
     'PP': 10,
     'is_combo': False,
     'is_contact': True},
    1: {'name': 'じしん',
     'priority': 0,
     'is_attack': True,
     'move_category': 'physical',
     'type': 'ground',
     '